<img src="images/splash3.png">
Requests data for the past hour and inserts this into SQL. To be ran hourly.


<p><b>What you were doing:</b></p>
<p>&nbsp; &nbsp; only write new tweets, in bottom cell </p>
<p>&nbsp; &nbsp; possible issue: only fetches tweets with 140+ character length? </p>

## Setup

In [272]:
import sqlite3 as lite
import sys
import requests
import pandas as pd
import numpy as np
import datetime
import json
import tweepy

In [273]:
# Twitter stuff
CONSUMER_KEY = 'hiZmnSOV63GdqWPqW5MN533T1'# TWITTER_CONSUMER_TOKEN
CONSUMER_SECRET = 'GJa7snhSw8jDQxTVhvoHJyIyJZBjLQikAeuNL0Y3o0Aw61zuOr' # TWITTER_CONSUMER_SECRET
ACCESS_TOKEN = '933287551212904448-uWflGB32dNF8rbgmBxJk1ql5AqemO14' # TWITTER_ACCESS_TOKEN
ACCESS_SECRET = 'boUb7syKFxrCmaJN3T3SHsWu6rDQQfdPxertNIEnXR6X9' # TWITTER_ACCESS_SECRET

## Class & function definitions


In [301]:
class Coin:
    def __init__(self, symbol, comparison_symbol, limit, aggregate):
        self.symbol = symbol
        self.comparison_symbol = comparison_symbol
        self.limit = limit
        self.aggregate = aggregate
        
    def fetch_marketdata(self):    
        url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
                .format(self.symbol.upper(), self.comparison_symbol.upper(), self.limit, self.aggregate)       
        page = requests.get(url)
        data = page.json()['Data']
        df = pd.DataFrame(data)
        df['mid'] = df[["high", "low"]].mean(axis=1)
        df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
        return df
    
    def fetch_tweets(self):
        # Create extractor object
        extractor = twitter_setup()
        # Create a tweet list
        #tweets = extractor.search(q ="${}".format(coin_instances[coin].symbol), count=20, tweet_mode="extended", lang="en")
        #print("[coin] = ", coin)
        #print("Searching for coin_instances[coin].symbol: ", coin_instances[coin].symbol)
        tweets = extractor.search(q ="${} {}".format(coin_instances[coin].symbol, "-url:binance -url:kucoin -url:ku-coin -url:discord"), 
                                  count=100, 
                                  tweet_mode="extended", lang="en")
        
        def fulltext(tweet):
            try:
                text = tweet.retweeted_status.full_text
                return(text)
            except:
                text = tweet.full_text
                return(text)

        df = pd.DataFrame(data=[fulltext(tweet) for tweet in tweets], columns=['Tweets'])
        
        df['time'] = np.array([tweet.created_at for tweet in tweets])
        df['time'] = df['time'].astype(np.int64) // 10**9 # Convert to unix
        df['ID']   = np.array([tweet.id for tweet in tweets])
        df['Len']  = np.array([len(tweet.full_text) for tweet in tweets])
        df['Reply_To_Status'] = np.array([tweet.in_reply_to_user_id for tweet in tweets])
        df['Reply_To_Name'] = np.array([tweet.in_reply_to_screen_name for tweet in tweets])
        df['Coordinates'] = np.array([tweet.coordinates for tweet in tweets])
        df['Retweet_Count'] = np.array([tweet.retweet_count for tweet in tweets])
        df['User_Verified'] = np.array([tweet.user.verified for tweet in tweets])
        df['User_Location'] = np.array([tweet.user.location for tweet in tweets])
        df['User_Followers_Count'] = np.array([tweet.user.followers_count for tweet in tweets])
        df['User_Favorite_Count'] = np.array([tweet.user.favourites_count for tweet in tweets])
        df['Source'] = np.array([tweet.source for tweet in tweets])
        df['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
        df['Favorite_Count'] = np.array([tweet.favorite_count for tweet in tweets])
        df['RTs'] = np.array([tweet.retweet_count for tweet in tweets])        
        return df
        
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication
    api = tweepy.API(auth)
    return api    
    
    
def market_to_db(market_data):
    con = None
    con = lite.connect('/hdd/db/crypto.db') #Connect to DB

    
    with con: #Dump to SQL
        date = pd.read_sql_query("SELECT time FROM {} ORDER BY time DESC LIMIT 1".format(coin_instances[coin].symbol), con)
        latest_db_entry = date['time'].values[0]
        if latest_db_entry < market_data.iloc[0]['time']:
            #print(latest_db_entry, "is lower than", market_data.iloc[0]['time'], "<- Time to write!") # Debug
            market_data.to_sql(name=coin_instances[coin].symbol, con=con, if_exists='append', index=True)
            #print("wrote to DB for coin", coin_instances[coin].symbol) # Debug 
        else:
            pass
            #print(latest_db_entry, "(latest db entry) is the same or higher than", market_data.iloc[0]['time'], "(fetched data), skipping write for", coinlist[coin])
            
def tweets_to_db(df_tweets):
    con = None
    con = lite.connect('/hdd/db/crypto.db') #Connect to DB
    
    with con: #Dump to SQL
        try:
            date = pd.read_sql_query("SELECT time FROM {}_tweets ORDER BY time DESC LIMIT 1".format(coin_instances[coin].symbol), con)
            latest_db_entry = date['time'].values[0] #Get last date
        except Exception as e:
            print("Skipping coin", coin_instances[coin].symbol, ", Error: ")
            print(e)
            latest_db_entry = 0 # Probably no table exists, set 
        
        df_tweets = df_tweets.loc[df_tweets['time'] > latest_db_entry] #Slice df to only include new tweets
        
        if(len(df_tweets) > 0): # If there is something to write 
            df_tweets.to_sql(name="{}_tweets".format(coin_instances[coin].symbol), con=con, if_exists='append', index=True)
            print("Going to write {} tweets to {}_tweets".format(len(df_tweets), coin_instances[coin].symbol))

## Fetch data & store in db

In [300]:
fh = open("/hdd/db/tracked_coins.json", 'r')
coinlist = json.load(fh)
coin_instances = [Coin(coinlist[coin], 'BTC','1','1') for coin in range(len(coinlist))] # Make Coin objects. Take all coins in coinlist, convert to BTC, poll 1 entry and aggregate per 1-hour period.

#Get data
for coin in range(len(coin_instances)):
    try:
        market_data = coin_instances[coin].fetch_marketdata() # Get coindata
        df_tweets = coin_instances[coin].fetch_tweets() # Get tweets
        market_data = market_data.drop(market_data.index[len(market_data)-1]) # Only keep the latest timestamped column
        market_to_db(market_data)
        tweets_to_db(df_tweets)
    except Exception as e:
        print("Skipping coin", coin_instances[coin].symbol, ", Error: ")
        print(e)

Going to write 13 tweets to ETH_tweets
Going to write 1 tweets to DGB_tweets
Going to write 1 tweets to LSK_tweets
Going to write 2 tweets to DGD_tweets
Going to write 1 tweets to WAVES_tweets
Going to write 2 tweets to ETC_tweets
Going to write 2 tweets to STRAT_tweets
Going to write 2 tweets to NEO_tweets
Going to write 1 tweets to XZC_tweets
Going to write 1 tweets to ZEC_tweets
Going to write 1 tweets to QTUM_tweets
Going to write 1 tweets to BAT_tweets
Going to write 1 tweets to BNT_tweets
Going to write 1 tweets to FUN_tweets
Going to write 1 tweets to SNT_tweets
Going to write 2 tweets to EOS_tweets
Going to write 1 tweets to GAS_tweets
Going to write 1 tweets to PPT_tweets
Going to write 2 tweets to BNB_tweets
Going to write 2 tweets to ZRX_tweets
Going to write 1 tweets to VEN_tweets
Going to write 1 tweets to WTC_tweets
Going to write 2 tweets to LRC_tweets
Going to write 2 tweets to TRX_tweets
Going to write 1 tweets to KNC_tweets
Going to write 2 tweets to SALT_tweets
Going

In [234]:
### JUST SOME PRINT LINES ###
coin = 98
print("Coin: ", coinlist[98])

market_data = coin_instances[98].fetch_marketdata() # Get coindata
print(market_data)

df_tweets = coin_instances[98].fetch_tweets() # Get tweets

Coin:  DNA
      close      high       low      open        time  volumefrom  volumeto  \
0  0.000009  0.000010  0.000009  0.000010  1531054800       500.6  0.004878   
1  0.000009  0.000009  0.000009  0.000009  1531058400         0.0  0.000000   

        mid           timestamp  
0  0.000010 2018-07-08 15:00:00  
1  0.000009 2018-07-08 16:00:00  
[coin] =  98
Searching for coin_instances[coin].symbol:  DNA


In [208]:
print("\n ------------- Random tweet -----------------:\n")
print(df_tweets['Tweets'].iloc[1])


 ------------- Random tweet -----------------:

@Crypto_Core Top 2-
$DONT
$JUMP

 a few others:
$BTC
$ETH
$DNA
$XMR
$ZRX
-in no order-

In 5 yr we’ll see a major shakeout of all these coins with no real purpose and $100’s MM valuations going broke, dumping 4$
Good purpose coins, &amp; &gt; market cap/solid standing- where it’s at :&gt;)


In [236]:
#Extract date of latest single db entry 
con = lite.connect('/hdd/db/crypto.db') #Connect to DB
date = pd.read_sql_query("SELECT time FROM {}_tweets ORDER BY time DESC LIMIT 1".format(coin_instances[4].symbol), con)
latest_db_entry = date['time'].values[0]
df_tweets = df_tweets.loc[df_tweets['time'] > latest_db_entry]
print(latest_db_entry)



1531061122


In [267]:

df_tweets2

,Tweets,Date,ID,Len,Reply_To_Status,Reply_To_Name,Coordinates,Retweet_Count,User_Verified,User_Location,User_Followers_Count,User_Favorite_Count,Source,Likes,Favorite_Count,RTs
0,Register with BitMEX for up to 100x Leverage!\...,1531065743,1015989481294442497,251,None,None,None,0,False,,38,0,623462453526465,0,0,0
1,Biggest Losers in the Last 24h 📉\n$GXS -7.1%\n...,1531065733,1015989440173477892,270,None,None,None,0,False,Crypto Aquarium,14128,15989,CryptoWhale,0,0,0
2,Biggest Losers in the Last 24h 📉\n$GXS -8.1%\n...,1531065637,1015989038057246720,140,None,None,None,1,False,,17,0,Twitter Web Client,0,0,1


In [237]:
#Find date of latest fetched tweet
df_tweets['time']
#print(df_tweets.loc[df_tweets['Date'] >= latest_db_entry])


# Reduce df_tweets to only new tweets 
for len(df_tweets) in range(df_tweets['time']:)
    select Date_of_last_row
     if Date_of_last_Row > latest_db_entry:
            slice df_tweets to only include the newer ones
            break

write 


            

0     1531061122
1     1531061013
2     1531061002
3     1531060836
4     1531060727
5     1531060672
6     1531060634
7     1531060518
8     1531059910
9     1531059640
10    1531058683
11    1531058597
12    1531058485
13    1531058400
14    1531058269
15    1531058012
16    1531057848
17    1531057822
18    1531057513
19    1531057445
Name: Date, dtype: int64

In [258]:
for row in df_tweets.itertuples():
    print (row.Date)

1531061122
1531061013
1531061002
1531060836
1531060727
1531060672
1531060634
1531060518
1531059910
1531059640
1531058683
1531058597
1531058485
1531058400
1531058269
1531058012
1531057848
1531057822
1531057513
1531057445


In [238]:
len(df_tweets)

20

In [111]:
latest_db_entry

Timestamp('2018-06-23 23:00:27')

In [114]:
# wat je moet doen: functie schrijven die dataframe sliced op timestamps die nieuwer zijn dan de latest_db_entry timestamp
# vind het punt en snij daar de helft af
# functie aanroepen in plaats van df_tweets.to_sql(name="{}_tweets.format)


df_tweets['time'] = pd.to_datetime(df_tweets['time'], unit = 's')

df_tweets

    

,Tweets,Date,ID,Len,Reply_To_Status,Reply_To_Name,Coordinates,Retweet_Count,User_Verified,User_Location,User_Followers_Count,User_Favorite_Count,Source,Likes,Favorite_Count,RTs
0,Buy this coin in Binance right now!\nPossible ...,2018-06-23 23:00:27,1010658875513671680,259,None,None,None,0,False,,60546,654,BinanceBEST024,0,0,0
1,Register with Binance and you can trade for as...,2018-06-23 23:00:20,1010658845113356288,265,None,None,None,0,False,,116,0,IFTTT,0,0,0
2,Bitcoin Cash $BCH price: $764.58 \n\nQUICK! Si...,2018-06-23 22:57:17,1010658076045832192,265,None,None,None,0,False,,37,0,modcfjodfjghfoijhioj,0,0,0
3,Current $WAVES price: $2.88 \n\nBinance is the...,2018-06-23 22:34:35,1010652364171300864,273,None,None,None,0,False,,15,0,wert3445983444982189,0,0,0
4,You still haven't registered at the #1 Crypto ...,2018-06-23 22:10:58,1010646423648468992,270,None,None,None,0,False,,3,0,Twitter Web Client,0,0,0
5,Replicate successful traders automatically wit...,2018-06-23 22:06:25,1010645278746013698,276,None,None,None,0,False,,1,0,Twitter Web Client,0,0,0
6,Lisk $LSK price: $5.58 \n\nBinance registratio...,2018-06-23 22:01:20,1010643997294039041,253,None,None,None,0,False,,211,5,twitterbotmaster21,0,0,0
7,Litecoin $LTC is valued at: $83.17 \n\nWe chec...,2018-06-23 22:01:00,1010643912304807936,262,None,None,None,0,False,,24,0,95fg9h5f9h29fgh9fg5h95,0,0,0
8,Register with Binance and you can trade for as...,2018-06-23 22:00:24,1010643764573081601,265,None,None,None,0,False,,116,0,IFTTT,0,0,0
9,Buy this coin in Binance right now!\nPossible ...,2018-06-23 22:00:23,1010643759854444544,259,None,None,None,0,False,,60546,654,BinanceBEST024,0,0,0


In [82]:
for row in df_tweets['time']:
    print (row)

1529794827
1529794820
1529794637
1529793275
1529791858
1529791585
1529791280
1529791260
1529791224
1529791223
1529790330
1529789803
1529789799
1529789709
1529788612
1529788397
1529787728
1529787619
1529787609
1529787517
